<a href="https://colab.research.google.com/github/daniela-estevez/ProyectoIntegrador/blob/main/Avance4_13Equipo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Avance 4. Modelos alternativos

## Equipo 13

## Alejandro García Hernández A01793812
## Daniela Estevez Rodriguez A01793723
## Carlos Alberto López Álvarez A01168193

## Objetivos

En esta etapa el objetivo es construir múltiples modelos con lafinalidad de explorar y evaluar cuál de ellos proporciona el mejor rendimiento para resolver el problema que nos fue asignado. El ejercicio se centra en la elección de los modelos, su entrenamiento y configuración de hiperparámetros para mejorar el desempeño.

## 0. Librerías

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Librerías a utilizar para el pre procesamiento.

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import LabelEncoder

In [3]:
# Librerías para la sección de modelos supervisados
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold, GridSearchCV, cross_val_predict
from sklearn.metrics import confusion_matrix,f1_score, accuracy_score, classification_report, recall_score, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.tree import export_graphviz
import graphviz
from IPython.display import display
from xgboost import plot_importance
from sklearn.preprocessing import LabelEncoder

In [4]:
# Librerías adicionales para la sección de modelos no-supervisados
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score

In [5]:
# El siguiente bloque se agregó para poder leer los archivos drive
#from google.colab import drive
#drive.mount('/content/drive')
#%cd /content/drive/MyDrive/Colab Notebooks/Proyecto Integrador

## 1. Preparación de datos

Se aplica el pre procesamiento de los datos que se ha trabajado en semanas anteriores

In [6]:
data = pd.read_csv("base_1.csv",sep=",", encoding="latin1")
df= data.copy()

In [7]:
# Eliminamos entonces estas características
df = df.drop(["Nombre","residencia","fecha_nacimiento","fecha_1er_visita","FECHA_INICIO_ACTUAL_EPISODIO"], axis=1)
df = df.dropna(axis=1, how='all')

In [8]:
# Calculamos el porcentaje de valores nulos en cada columna
null_percentage = (df.isnull().sum() / len(df)) * 100

# Con el sguiente código, se crea un DataFrame con los porcentajes de valores nulos
null_df = pd.DataFrame({'Columna': null_percentage.index, 'Porcentaje_Null': null_percentage.values})

# Filtramos el DataFrame para mantener solo las columnas donde el porcentaje de valores nulos sea menor al 40%
columnas_a_mantener = null_df[null_df['Porcentaje_Null'] < 40]['Columna']

# Filtramos el DataFrame original para mantener solo las columnas que queremos conservar
df = df[columnas_a_mantener]

In [9]:
#Primero obtenemos las variables que reportan un solo valor y mostramos sus nombres
unique_counts = df.nunique()
columnas_a_eliminar = unique_counts[unique_counts == 1].index
columnas_a_eliminar
df = df.drop(columns=columnas_a_eliminar)

In [10]:
binary_columns = []
for column in df.columns:
    unique_values = df[column].unique()
    if len(unique_values) == 2 and all(value in [0, 1] for value in unique_values):
        binary_columns.append(column)

# Número de columnas con solamente 0's o 1's
print("Numero de columnas binarias encontradas:", len(binary_columns))

Numero de columnas binarias encontradas: 28


In [11]:
# Actualizamos el tipo de las columnas binarias a booleano usando astype()
for column in binary_columns:
    df[column] = df[column].astype(bool)

In [12]:
# Lista de variables que son en realidad categóricas
cat_cols=["SEXO","CONDICION_ACTUAL","ESTADO_civil","RELIGION","PROVEEDOR_FAMILIAR","OCUPACIoN_JEFE_FAMILIA","ESCOLARIDAD__JEFE_FAMILIA",
         "ESCOLARIDAD_MAXIMA_PX","DX_PRIMARIO","CODIGO_DX.1","TRASTORNO_MAYOR_DIAGNOSTICO",
         "Riesgo_suicidio","Sintomas_ansiosos"]

# Actualizamos el tipo de las columnas categoricas usando astype()
for column in cat_cols:
    df[column] = df[column].astype("category")

In [13]:
# Actualizamos el tipo de la columna MENARCA a uno numérico usando astype()
df["MENARCA"] = df["MENARCA"].astype("float64")

In [14]:
# Sustituir los valores de MENARCA donde SEXO es "M"
df.loc[df['SEXO'] == 'M', 'MENARCA'] = -1

# Imputar la moda a las variables numéricas
for columna in df.select_dtypes(include='number').columns:
    df[columna] = df[columna].fillna(df[columna].median())

# Imputar la moda a las variables booleanas
for columna in df.select_dtypes(include='bool').columns:
    moda = df[columna].mode()[0]  # Calcular la moda
    df[columna] = df[columna].fillna(moda)

# Imputar la moda a las variables alfanuméricas
for columna in df.select_dtypes(include='category').columns:
    moda = df[columna].mode()[0]  # Calcular la moda
    df[columna] = df[columna].fillna(moda)

In [15]:
# Lista de variables que son en realidad enteros
enteros_cols=["EDAD_1era_visita","EDAD_INICIO_1er_EPISODIO","AnioS_ESTUDIO_PACIENTE","MENARCA","INICIO_VIDA_SEXUAL_ACTIVA","MADRS_TOTAL","CGI-S.1"]

# Actualizamos el tipo de las columnas de enteros usando astype()
for column in enteros_cols:
    df[column] = df[column].astype("int64")

In [16]:
# Eliminamos estas características
df = df.drop(["EPISODIO_MAYOR_total_vida","SUICIDALIDAD_ACTUAL","SUICIDALIDAD_INTENTO_total_vida",
              "B1b_INTENCION_DE_MORIR_EN_ACCIDENTE","B5_PENSO_METODO_SUICIDARSE","B8_PENSO_FECHA_SUICIDIO",
              "B10_INTENCION_SUICIDIO","B18_INTENTO_SUICIDARSE","B2_NECESIDAD_DE_ESTAR_MUERTO","TRASTORNO_MAYOR_PASADO"],
             axis=1)

In [17]:
# Eliminamos estas características
df = df.drop(["CODIGO_DX.1","Puntaje_experto"],
             axis=1)

In [18]:
# Reemplazar el valor 1955 por la mediana
df['EDAD_INICIO_1er_EPISODIO'] = df['EDAD_INICIO_1er_EPISODIO'].replace(1955, df['EDAD_INICIO_1er_EPISODIO'].median())

# Reemplazar el valor 1 por la mediana
df['INICIO_VIDA_SEXUAL_ACTIVA'] = df['INICIO_VIDA_SEXUAL_ACTIVA'].replace(1, df['INICIO_VIDA_SEXUAL_ACTIVA'].median())

In [19]:
# Realizamos una copia del df para realizar una trasnformación
df_T=df.copy()

**Winsonorización:** Reemplaza los valores atípicos con valores en el percentil p-ésimo o (1 - p)-ésimo para reducir su impacto sin eliminarlos.

In [20]:
# Transformamos las variables usando Winsonorización
df_T['INICIO_VIDA_SEXUAL_ACTIVA_W'] = winsorize(df_T['INICIO_VIDA_SEXUAL_ACTIVA'], limits=(0.05, 0.05))
df_T['DURACION_EPISODIO_MAYOR_ACTUAL_W'] = winsorize(df_T['DURACION_EPISODIO_MAYOR_ACTUAL'], limits=(0.05, 0.05))

In [21]:
# Eliminamos estas características originales
df_T = df_T.drop(["INICIO_VIDA_SEXUAL_ACTIVA","DURACION_EPISODIO_MAYOR_ACTUAL"],
             axis=1)

In [22]:
# Hacemos listas de las variables ordinales
cat_orginales= ['ESCOLARIDAD__JEFE_FAMILIA', 'ESCOLARIDAD_MAXIMA_PX', 'DX_PRIMARIO','TRASTORNO_MAYOR_DIAGNOSTICO',
                'Riesgo_suicidio','Sintomas_ansiosos']

In [23]:
# Creamos una copia del dataframe
df_TC=df_T.copy()

In [24]:
# Crear el encoder ordinal
encoder_1 = OrdinalEncoder(categories=[["Menos de 6 años", "Primaria", "Secundaria", "Preparatoria",
                                     "Estudios universitarios no terminados", "Licenciatura", "Posgrado"]], dtype=int)
encoder_2 = OrdinalEncoder(categories=[["Distimia","Trastorno depresivo episodio único ",
                                        "Trastorno depresivo mayor recidivante"]], dtype=int)
encoder_3 = OrdinalEncoder(categories=[["0","TRASTORNO DEPRESIVO MAYOR EPISODIO UNICO",
                                        "TRASTORNO DEPRESIVO MAYOR PRIMER EPISODIO",
                                        "TRASTORNO DEPRESIVO MAYOR RECURRENTE",
                                        "TRASTORNO DEPRESIVO MAYOR RECIDIVANTE ",
                                        "TRASTORNO DEPRESIVO MAYOR RECURRENTE RESISTENTE",
                                        "TRASTORNO DEPRESIVO MAYOR RECURRENTE SEVERO",
                                        "TRASTORNO DEPRESIVO MAYOR"]], dtype=int)
encoder_4 = OrdinalEncoder(categories=[["Leve","Moderado","Alto"]], dtype=int)
encoder_5 = OrdinalEncoder(categories=[["Leve","Moderado-Grave"]], dtype=int)
encoder_6 = OrdinalEncoder(categories=[["Primera aparición sin antecedente","Indistinguible del pasado",
                                        "Recurrencia condición previa",
                                        "Significativamente diferente de condición previa",
                                        "Exageración de trastorno crónico"]], dtype=int)

# Ajustar y transformar los datos
df_TC['ESCOLARIDAD__JEFE_FAMILIA_encoded'] = encoder_1.fit_transform(df_TC[['ESCOLARIDAD__JEFE_FAMILIA']])
df_TC['ESCOLARIDAD_MAXIMA_PX_encoded'] = encoder_1.fit_transform(df_TC[['ESCOLARIDAD_MAXIMA_PX']])
df_TC['DX_PRIMARIO_encoded'] = encoder_2.fit_transform(df_TC[['DX_PRIMARIO']])
df_TC['TRASTORNO_MAYOR_DIAGNOSTICO_encoded'] = encoder_3.fit_transform(df_TC[['TRASTORNO_MAYOR_DIAGNOSTICO']])
df_TC['Riesgo_suicidio_encoded'] = encoder_4.fit_transform(df_TC[['Riesgo_suicidio']])
df_TC['Sintomas_ansiosos_encoded'] = encoder_5.fit_transform(df_TC[['Sintomas_ansiosos']])
df_TC['CONDICION_ACTUAL_encoded'] = encoder_6.fit_transform(df_TC[['CONDICION_ACTUAL']])

In [25]:
# Eliminamos estas características originales
df_TC = df_TC.drop(["ESCOLARIDAD__JEFE_FAMILIA","ESCOLARIDAD_MAXIMA_PX","DX_PRIMARIO","TRASTORNO_MAYOR_DIAGNOSTICO",
                "Riesgo_suicidio","Sintomas_ansiosos","CONDICION_ACTUAL"],
             axis=1)

In [26]:
# Hacemos listas de las variables no ordinales
cat_no_orginales= ['SEXO', 'ESTADO_civil', 'RELIGION','PROVEEDOR_FAMILIAR', 'OCUPACIoN_JEFE_FAMILIA']

In [27]:
# Creamos una copia del ultimo dataframe
df_TCL=df_TC.copy()

In [28]:
# Inicializar el LabelEncoder
encoder = LabelEncoder()

# Iterar sobre las columnas y aplicar el LabelEncoder
for col in cat_no_orginales:
    df_TCL[col] = encoder.fit_transform(df_TCL[col])

In [29]:
# Creamos una copia del ultimo dataframe
df_TC_encoded=df_TC.copy()

In [30]:
# Aplicar Get_dummies asegurando de eliminar la primera categoría
df_TC_encoded = pd.get_dummies(df_TC_encoded, columns=cat_no_orginales, drop_first=True)

## 2 Modelos supervisados

Comanzaremos con una observación importante, el proyecto con el que estamos trabajando tiene que ver con la salud mental y física de pacientes, por lo que la métrica más adecuada para nuestros fines será **RECALL**, con el objetivo de evitar los falsos negativos en la medida de lo posible. No obstante, también mostraremos la métrica F1 como referencia para que, de ser posible y sin ser el objetivo principal, se minimicen los falsos positivos.

Para esta etapa del proyecto profundizamos un poco más en los 7 modelos supervisados que elegimos probar.

* **Regresión logística.** Este es un modelo simple y fácil de interpretar. Además, el modelo obtiene las probabilidades de pertenencia a cada clase, lo que puede servir para realizar umbrales más laxos o reestrictivos.
* **SVM.** Este algoritmo funciona bien aún con un número grande de características y, aunque la base con dummies no es gigante, al final tenemos 69 variables.
* **KNN.** Fácil entender e implementar.
* **Árbol de Decisión.** Elefimos este porque es muy fácil de interpretar y se puede visualizar, además de que funciona bastante bien sin la necesidad de realizar tantas transformaciones a los datos, lo que muchas veces dificulta la interpretación.
* **Bosque Aleatorio.** Al realizar la agregación de múltiples árboles de decisión, ayuda a reducir el riesgo de sobreajuste.
* **Red neuronal.** Este algoritmo es potente para problemas con relaciones no lineales y complejas al descubrir patrones complejos. No obstante, su interpretación no es transparente.
* **XGBoost.** Este algoritmo suele obtener una precisión muy alta y con el uso de boosting evita el sobreajuste y mejorara la generalización en datos no vistos. Gracias a lo anterior con este algoritmo se han ganado muchas competiciones de machine learning.

Dividimos los modelos anteriores en dos grupos, ya que algunos de ellos necesitan que los datos tengan algunas transformaciones particulares para poder procesar y modelar adecuadamente la información de las variables categóricas en sus cálculos y predicciones.

### 2.1 Regresión logística, SVM y KNN

Para estos modelos utilizaremos la base con variables dummies, observamos que esta base consta de 69 variables y 380 observaciones

In [ ]:
print(f"Dataframe usando get dummies tiene {df_TC_encoded.shape[1]} columnas.")
print(f"Dataframe usando get dummies tiene {df_TC_encoded.shape[0]} renglones.")

Dataframe usando get dummies tiene 69 columnas.
Dataframe usando get dummies tiene 380 renglones.


In [ ]:
#Generamos la base con las características y la variable de salida
X1 = df_TC_encoded.drop(columns=['Sintomas_ansiosos_encoded'])
Y1 = df_TC_encoded['Sintomas_ansiosos_encoded']

Aunque nuestra base es pequeña, debemos de dividirla para obtener datos de entrenamiento y validación. Así, usaremos el 90% de los datos para entrenar los modelos.

In [ ]:
Xtv1, Xtest1, ytv1, ytest1 = train_test_split(X1, Y1, train_size = 0.9, random_state = 0)

#### 2.1.1 Regresión logística

In [ ]:
# Definimos el modelo
reg = LogisticRegression(solver='liblinear', class_weight= "balanced", random_state=0)

# Vamos a probar estos hiperparámetros
param_grid = {
    'penalty': ['l1', 'l2', 'none'],
    #'C': [0.001, 0.01, 0.1],
    'max_iter': [100, 200, 300],
    'tol': [1e-4, 1e-3, 1e-2, 1e-1]
}

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=0)

grid = GridSearchCV(estimator=reg,
                  param_grid=param_grid,
                  cv=cv,
                  scoring="recall")

# se ajusta el modelo
grid.fit(Xtv1, ytv1)

# Imprimimos los mejores hiperparámetros encontrados
print("Mejores parámetros encontrados:", grid.best_params_)


# Obtenemos el rendimiento del modelo con el conjunto de entrenamiento usando validación cruzada
ytv1_pred = cross_val_predict(grid.best_estimator_, Xtv1, ytv1, cv=5)

accuracy_tv = accuracy_score(ytv1, ytv1_pred)
recall_tv = recall_score(ytv1, ytv1_pred, average='binary')
f1_tv = f1_score(ytv1, ytv1_pred, average='binary')
cm_tv = confusion_matrix(ytv1, ytv1_pred)

print("Resultados en el conjunto de validación:")
print(f"Accuracy: {accuracy_tv:.4f}")
print(f"Recall: {recall_tv:.4f}")
print(f"F1 Score: {f1_tv:.4f}")
print(f'Confusion Matrix:\n{cm_tv}\n')

# ahora obtenemos el resultado con el conjunto de prueba
ytest1_pred = grid.best_estimator_.predict(Xtest1)

accuracy_test = accuracy_score(ytest1, ytest1_pred)
recall_test = recall_score(ytest1, ytest1_pred, average='binary')
f1_test = f1_score(ytest1, ytest1_pred, average='binary')
cm_test = confusion_matrix(ytest1, ytest1_pred)

print("Resultados en el conjunto de prueba:")
print(f"Accuracy: {accuracy_test:.4f}")
print(f"Recall: {recall_test:.4f}")
print(f"F1 Score: {f1_test:.4f}")
print(f'Confusion Matrix:\n{cm_test}\n')

Mejores parámetros encontrados: {'max_iter': 100, 'penalty': 'l1', 'tol': 0.0001}
Resultados en el conjunto de validación:
Accuracy: 0.5731
Recall: 0.5973
F1 Score: 0.6439
Confusion Matrix:
[[ 64  57]
 [ 89 132]]

Resultados en el conjunto de prueba:
Accuracy: 0.6053
Recall: 0.6250
F1 Score: 0.6667
Confusion Matrix:
[[ 8  6]
 [ 9 15]]



#### 2.1.2 SVM

Tardó 28 minutos con los parámetros que están comentados y así me da recall de 1...

In [ ]:
# Definimos el modelo
svm = SVC(class_weight='balanced',random_state=0)

# Vamos a probar estos hiperparámetros
param_grid = {
    'C': [0.001, 0.01, 0.1],
#    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'gamma': [0.001, 0.01, 0.1],
#    'degree': [2, 3, 4]
}

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=0)

grid = GridSearchCV(estimator=svm,
                  param_grid=param_grid,
                  cv=cv,
                  scoring="recall")

# se ajusta el modelo
grid.fit(Xtv1, ytv1)

# Imprimimos los mejores hiperparámetros encontrados
print("Mejores parámetros encontrados:", grid.best_params_)


# Obtenemos el rendimiento del modelo con el conjunto de entrenamiento usando validación cruzada
ytv1_pred = cross_val_predict(grid.best_estimator_, Xtv1, ytv1, cv=5)

accuracy_tv = accuracy_score(ytv1, ytv1_pred)
recall_tv = recall_score(ytv1, ytv1_pred, average='binary')
f1_tv = f1_score(ytv1, ytv1_pred, average='binary')
cm_tv = confusion_matrix(ytv1, ytv1_pred)

print("Resultados en el conjunto de validación:")
print(f"Accuracy: {accuracy_tv:.4f}")
print(f"Recall: {recall_tv:.4f}")
print(f"F1 Score: {f1_tv:.4f}")
print(f'Confusion Matrix:\n{cm_tv}\n')

# ahora obtenemos el resultado con el conjunto de prueba
ytest1_pred = grid.best_estimator_.predict(Xtest1)

accuracy_test = accuracy_score(ytest1, ytest1_pred)
recall_test = recall_score(ytest1, ytest1_pred, average='binary')
f1_test = f1_score(ytest1, ytest1_pred, average='binary')
cm_test = confusion_matrix(ytest1, ytest1_pred)

print("Resultados en el conjunto de prueba:")
print(f"Accuracy: {accuracy_test:.4f}")
print(f"Recall: {recall_test:.4f}")
print(f"F1 Score: {f1_test:.4f}")
print(f'Confusion Matrix:\n{cm_test}\n')

Mejores parámetros encontrados: {'C': 0.001, 'gamma': 0.001, 'kernel': 'linear'}
Resultados en el conjunto de validación:
Accuracy: 0.6023
Recall: 0.6561
F1 Score: 0.6808
Confusion Matrix:
[[ 61  60]
 [ 76 145]]

Resultados en el conjunto de prueba:
Accuracy: 0.5263
Recall: 0.5417
F1 Score: 0.5909
Confusion Matrix:
[[ 7  7]
 [11 13]]



#### 2.1.3 KNN

In [ ]:
# Definimos el modelo
knn = KNeighborsClassifier()

# Vamos a probar estos hiperparámetros
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=0)

grid = GridSearchCV(estimator=knn,
                  param_grid=param_grid,
                  cv=cv,
                  scoring="recall")

# se ajusta el modelo
grid.fit(Xtv1, ytv1)

# Imprimimos los mejores hiperparámetros encontrados
print("Mejores parámetros encontrados:", grid.best_params_)


# Obtenemos el rendimiento del modelo con el conjunto de entrenamiento usando validación cruzada
ytv1_pred = cross_val_predict(grid.best_estimator_, Xtv1, ytv1, cv=5)

accuracy_tv = accuracy_score(ytv1, ytv1_pred)
recall_tv = recall_score(ytv1, ytv1_pred, average='binary')
f1_tv = f1_score(ytv1, ytv1_pred, average='binary')
cm_tv = confusion_matrix(ytv1, ytv1_pred)

print("Resultados en el conjunto de validación:")
print(f"Accuracy: {accuracy_tv:.4f}")
print(f"Recall: {recall_tv:.4f}")
print(f"F1 Score: {f1_tv:.4f}")
print(f'Confusion Matrix:\n{cm_tv}\n')

# ahora obtenemos el resultado con el conjunto de prueba
ytest1_pred = grid.best_estimator_.predict(Xtest1)

accuracy_test = accuracy_score(ytest1, ytest1_pred)
recall_test = recall_score(ytest1, ytest1_pred, average='binary')
f1_test = f1_score(ytest1, ytest1_pred, average='binary')
cm_test = confusion_matrix(ytest1, ytest1_pred)

print("Resultados en el conjunto de prueba:")
print(f"Accuracy: {accuracy_test:.4f}")
print(f"Recall: {recall_test:.4f}")
print(f"F1 Score: {f1_test:.4f}")
print(f'Confusion Matrix:\n{cm_test}\n')

Mejores parámetros encontrados: {'algorithm': 'auto', 'metric': 'euclidean', 'n_neighbors': 9, 'weights': 'uniform'}
Resultados en el conjunto de validación:
Accuracy: 0.6053
Recall: 0.8235
F1 Score: 0.7295
Confusion Matrix:
[[ 25  96]
 [ 39 182]]

Resultados en el conjunto de prueba:
Accuracy: 0.5526
Recall: 0.7500
F1 Score: 0.6792
Confusion Matrix:
[[ 3 11]
 [ 6 18]]



### 2.2 Árbol de decisión, Bosque aleatorio, Red neuronal, XGBoost

Ahora probaremos con otro tipo de modelos en los que no es necesario utilizar variables dummies. Observamos que esta base consta de 39 variables y 380 observaciones

In [31]:
print(f"Dataframe usando label ecoding tiene {df_TCL.shape[1]} columnas.")
print(f"Dataframe usando label ecoding tiene {df_TCL.shape[0]} renglones.")

Dataframe usando label ecoding tiene 39 columnas.
Dataframe usando label ecoding tiene 380 renglones.


In [32]:
#Generamos la base con las características y la variable de salida
X2 = df_TCL.drop(columns=['Sintomas_ansiosos_encoded'])
Y2 = df_TCL['Sintomas_ansiosos_encoded']

Como ya explicamos anteriormente, utilizaremos el 90% de los datos para el entrenamiento además de validación cruzada.

In [33]:
Xtv2, Xtest2, ytv2, ytest2 = train_test_split(X2, Y2, train_size = 0.9, random_state = 0)

#### 2.2.1 Árbol de decisión

In [ ]:
# Definimos el modelo
arbol = DecisionTreeClassifier(random_state=0)

# Vamos a probar estos hiperparámetros
param_grid = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [7, 10, 15],
    'min_samples_leaf': [7, 10, 15],
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion': ['gini', 'entropy']
}

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=0)

grid = GridSearchCV(estimator=arbol,
                  param_grid=param_grid,
                  cv=cv,
                  scoring="recall")

# se ajusta el modelo
grid.fit(Xtv2, ytv2)

# Imprimimos los mejores hiperparámetros encontrados
print("Mejores parámetros encontrados:", grid.best_params_)


# Obtenemos el rendimiento del modelo con el conjunto de entrenamiento usando validación cruzada
ytv2_pred = cross_val_predict(grid.best_estimator_, Xtv2, ytv2, cv=5)

accuracy_tv = accuracy_score(ytv2, ytv2_pred)
recall_tv = recall_score(ytv2, ytv2_pred, average='binary')
f1_tv = f1_score(ytv2, ytv2_pred, average='binary')
cm_tv = confusion_matrix(ytv2, ytv2_pred)

print("Resultados en el conjunto de validación:")
print(f"Accuracy: {accuracy_tv:.4f}")
print(f"Recall: {recall_tv:.4f}")
print(f"F1 Score: {f1_tv:.4f}")
print(f'Confusion Matrix:\n{cm_tv}\n')

# ahora obtenemos el resultado con el conjunto de prueba
ytest2_pred = grid.best_estimator_.predict(Xtest2)

accuracy_test = accuracy_score(ytest2, ytest2_pred)
recall_test = recall_score(ytest2, ytest2_pred, average='binary')
f1_test = f1_score(ytest2, ytest2_pred, average='binary')
cm_test = confusion_matrix(ytest2, ytest2_pred)

print("Resultados en el conjunto de prueba:")
print(f"Accuracy: {accuracy_test:.4f}")
print(f"Recall: {recall_test:.4f}")
print(f"F1 Score: {f1_test:.4f}")
print(f'Confusion Matrix:\n{cm_test}\n')

Mejores parámetros encontrados: {'criterion': 'entropy', 'max_depth': 3, 'max_features': 'auto', 'min_samples_leaf': 10, 'min_samples_split': 7}
Resultados en el conjunto de validación:
Accuracy: 0.6082
Recall: 0.9276
F1 Score: 0.7537
Confusion Matrix:
[[  3 118]
 [ 16 205]]

Resultados en el conjunto de prueba:
Accuracy: 0.5789
Recall: 0.9167
F1 Score: 0.7333
Confusion Matrix:
[[ 0 14]
 [ 2 22]]



#### 2.2.2 Bosque aleatorio

In [ ]:
# Definimos el modelo
bosque = RandomForestClassifier(random_state=0)

# Vamos a probar estos hiperparámetros
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [7, 10, 15],
    'min_samples_leaf': [7, 10, 15],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy']
}

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=0)

grid = GridSearchCV(estimator=bosque,
                  param_grid=param_grid,
                  cv=cv,
                  scoring="recall")

# se ajusta el modelo
grid.fit(Xtv2, ytv2)

# Imprimimos los mejores hiperparámetros encontrados
print("Mejores parámetros encontrados:", grid.best_params_)


# Obtenemos el rendimiento del modelo con el conjunto de entrenamiento usando validación cruzada
ytv2_pred = cross_val_predict(grid.best_estimator_, Xtv2, ytv2, cv=5)

accuracy_tv = accuracy_score(ytv2, ytv2_pred)
recall_tv = recall_score(ytv2, ytv2_pred, average='binary')
f1_tv = f1_score(ytv2, ytv2_pred, average='binary')
cm_tv = confusion_matrix(ytv2, ytv2_pred)

print("Resultados en el conjunto de validación:")
print(f"Accuracy: {accuracy_tv:.4f}")
print(f"Recall: {recall_tv:.4f}")
print(f"F1 Score: {f1_tv:.4f}")
print(f'Confusion Matrix:\n{cm_tv}\n')

# ahora obtenemos el resultado con el conjunto de prueba
ytest2_pred = grid.best_estimator_.predict(Xtest2)

accuracy_test = accuracy_score(ytest2, ytest2_pred)
recall_test = recall_score(ytest2, ytest2_pred, average='binary')
f1_test = f1_score(ytest2, ytest2_pred, average='binary')
cm_test = confusion_matrix(ytest2, ytest2_pred)

print("Resultados en el conjunto de prueba:")
print(f"Accuracy: {accuracy_test:.4f}")
print(f"Recall: {recall_test:.4f}")
print(f"F1 Score: {f1_test:.4f}")
print(f'Confusion Matrix:\n{cm_test}\n')

Mejores parámetros encontrados: {'bootstrap': True, 'criterion': 'gini', 'max_depth': 3, 'max_features': 'auto', 'min_samples_leaf': 15, 'min_samples_split': 7, 'n_estimators': 100}
Resultados en el conjunto de validación:
Accuracy: 0.6374
Recall: 0.9729
F1 Score: 0.7762
Confusion Matrix:
[[  3 118]
 [  6 215]]

Resultados en el conjunto de prueba:
Accuracy: 0.6579
Recall: 0.9583
F1 Score: 0.7797
Confusion Matrix:
[[ 2 12]
 [ 1 23]]



#### 2.2.3 Red neuronal

In [ ]:
# Definimos el modelo
red = MLPClassifier(solver="adam", random_state=0)

# Vamos a probar estos hiperparámetros
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50), (100, 100)],
    'activation': ['logistic', 'tanh', 'relu'],
    'alpha': [0.001, 0.01, 0.1],
    'learning_rate_init': [0.001, 0.01, 0.1],
    'max_iter': [200, 400, 600]
}

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=0)

grid = GridSearchCV(estimator=red,
                  param_grid=param_grid,
                  cv=cv,
                  scoring="recall")

# se ajusta el modelo
grid.fit(Xtv2, ytv2)

# Imprimimos los mejores hiperparámetros encontrados
print("Mejores parámetros encontrados:", grid.best_params_)


# Obtenemos el rendimiento del modelo con el conjunto de entrenamiento usando validación cruzada
ytv2_pred = cross_val_predict(grid.best_estimator_, Xtv2, ytv2, cv=5)

accuracy_tv = accuracy_score(ytv2, ytv2_pred)
recall_tv = recall_score(ytv2, ytv2_pred, average='binary')
f1_tv = f1_score(ytv2, ytv2_pred, average='binary')
cm_tv = confusion_matrix(ytv2, ytv2_pred)

print("Resultados en el conjunto de validación:")
print(f"Accuracy: {accuracy_tv:.4f}")
print(f"Recall: {recall_tv:.4f}")
print(f"F1 Score: {f1_tv:.4f}")
print(f'Confusion Matrix:\n{cm_tv}\n')

# ahora obtenemos el resultado con el conjunto de prueba
ytest2_pred = grid.best_estimator_.predict(Xtest2)

accuracy_test = accuracy_score(ytest2, ytest2_pred)
recall_test = recall_score(ytest2, ytest2_pred, average='binary')
f1_test = f1_score(ytest2, ytest2_pred, average='binary')
cm_test = confusion_matrix(ytest2, ytest2_pred)

print("Resultados en el conjunto de prueba:")
print(f"Accuracy: {accuracy_test:.4f}")
print(f"Recall: {recall_test:.4f}")
print(f"F1 Score: {f1_test:.4f}")
print(f'Confusion Matrix:\n{cm_test}\n')

### 2.3 Conclusiones

Al ajustar los hiperparámetros de los 7 modelos supervisados elegidos, obtivimos diferentes resultados a los de la entrega anterior. Si bien, en algunos casos el rendimiento cayó y en algunos otros fue mejor (el rendimiento obtenido la semana previa sin el refinamiento de hiperparámetros se mostrará entre paréntesis), en todos los modelos notamos un mucho mejor comportamiento en cuanto al sobreajuste.

Así, al realizar la busqueda de algunos de los mejores hiperparámetros:

* Para la regresión logística obtuvimos un recall de 60% y 62% con el conjunto de entrenamiento y el de validación respectivamente (75% en la entrega anterior). Si bien, se esperaría que el rendimiento fuera mejor en el conjunto de entrenamiento, una diferencia del 2% es relativamente pequeña y podría estar dentro del margen de error.

* En cuanto a SVM obtenemos 65% en el entrenamiento (60% en la entrega anterior), y aún habiendo probado hiperparámetros para intentar evitar el sobreajuste, al utilizar los datos de prueba notamos que el rendimiento bajó considerablemente por lo que el modelo parece estar sobreentrenado.

* Con KNN obtuvimos 82% y 75% de rendimiento con el conjunto de entrenamiento y el de validación respectivamente (75% y 62% en la entrega anterior). Aunque obtuvimos mucho mejores resultados al refinar los hiperparámetros, aún tenemos un problema de sobreentrenamiento, si bien mucho menor que la semana anterior.

* Con árboles de decisión obtuvimos una recall de 93% y 92%, con los conjuntos de entrenamiento y prueba, respectivamente (70% y 75% en la entrega anterior). Además el F1 también es bastante aceptable 75% con el conjunto de entrenamiento y 73% con el de prueba. Lo anterior, representa una mejoría considerable con respecto al rendimiento obtenido la semana previa, en donde además, teníamos sobreajuste.

* Con el bosque aleatorio tuvimos resultados ligeramente mejores al árbol de decisión. Obtuvimos un recall de 97% en el entrenamiento y 96% con el conjunto de prueba (83% y 87% en la entrega anterior), y un F1 de 78% con ambos conjuntos de datos. Al igual que con el árbol de decisión, al refinar los hiperparámetros obtuvimos mejores resultados que la semana previa además de, al parecer, evitar el sobreajuste.
